In [15]:
import pandas as pd
import numpy as np

total = 1_427_843
total_ac = 625_781
total_nac = 802_062

In [16]:
f = '../data/002_intermediate/vivienda_energia_m2_estrato_clima.parquet'
vivienda = pd.read_parquet(f)
print("se tiran ",vivienda.m2.isna().sum())
print("Iniciales ",vivienda.factor.sum())
vivienda.dropna(inplace=True)
print("Restan    ",vivienda.factor.sum())


se tiran  51
Iniciales  1036944
Restan     951967


In [17]:
f = "../data/001_raw/ENCEVI_2018/aireacond.csv"
aire = pd.read_csv(f,index_col=0)
folios_con_ac = aire.index.unique()

# 2. Subconjunto de viv con AC
viv_con_ac = vivienda.loc[ vivienda.index.isin(folios_con_ac) ]

# 3. Subconjunto de viv sin AC
viv_sin_ac = vivienda.loc[ ~vivienda.index.isin(folios_con_ac) ]

# 4. Comprobar tamaños
print("Con AC:", viv_con_ac.factor.sum())
print("Sin AC:", viv_sin_ac.factor.sum())

Con AC: 419033
Sin AC: 532934


In [18]:
# viv_con_ac.to_parquet("../data/002_intermediate/vivienda_con_aa.parquet")
# viv_sin_ac.to_parquet("../data/002_intermediate/vivienda_sin_aa.parquet")

In [19]:
ac_mean = np.average(viv_con_ac['kwh_mes'],weights=viv_con_ac['factor'])
nac_mean = np.average(viv_sin_ac['kwh_mes'],weights=viv_sin_ac['factor'])

In [20]:
ac_mean, nac_mean

(np.float64(159.55143774011103), np.float64(120.54649414890874))

In [21]:
ac_energia = ac_mean*total_ac
nac_energia = nac_mean*total_nac
ac_energia, nac_energia

(np.float64(99844258.26044442), np.float64(96685762.19006203))

In [22]:
total_energia = ac_energia + nac_energia
total_energia

np.float64(196530020.45050645)

In [23]:
# Porcentaje de viviendas
porc_ac = total_ac / total * 100
porc_nac = total_nac / total * 100


# Porcentaje del consumo
porc_consumo_ac = ac_energia / total_energia * 100
porc_consumo_nac = nac_energia / total_energia * 100

# Crear tabla
import pandas as pd

tabla = pd.DataFrame({
    'Tipo de vivienda': ['Con AC', 'Sin AC'],
    'Viviendas (%)': [porc_ac, porc_nac],
    'Consumo (%)': [porc_consumo_ac, porc_consumo_nac]
})

print(tabla)


  Tipo de vivienda  Viviendas (%)  Consumo (%)
0           Con AC      43.827017    50.803566
1           Sin AC      56.172983    49.196434


## Calculo de con aire acondicionado

In [24]:
df = viv_con_ac[['m2','estrato','kwh_mes','factor']].copy()
df = df.dropna(subset=['kwh_mes','factor'])
df = df[df['factor'] > 0]

# Consumo ponderado por registro (kWh/mes * peso)
df['consumo_pond'] = df['kwh_mes'] * df['factor']

# Agregado por m2 y estrato
res = (
    df.groupby(['m2','estrato'], as_index=False)
      .agg(consumo_total_pond=('consumo_pond','sum'),
           peso=('factor','sum'),
           n_filas=('kwh_mes','size'))
)

# Porcentaje respecto al consumo total (suma ponderada global)
total_consumo = res['consumo_total_pond'].sum()
res['pct_consumo_total'] = 100 * res['consumo_total_pond'] / total_consumo

# Orden y redondeo para presentación
res = res.sort_values(['estrato','m2']).reset_index(drop=True)
res['consumo_total_pond'] = res['consumo_total_pond'].round(2)
res['pct_consumo_total'] = res['pct_consumo_total'].round(2)

res


,m2,estrato,consumo_total_pond,peso,n_filas,pct_consumo_total
0,55.0,Alto,404776.30,5232,3,0.61
1,75.0,Alto,1587764.40,11841,7,2.37
2,100.0,Alto,8105589.07,59321,37,12.12
3,150.0,Alto,7213327.21,43246,27,10.79
4,200.0,Alto,5224366.26,19064,11,7.81
5,55.0,Bajo,87317.15,1641,2,0.13
6,75.0,Bajo,168482.71,2581,3,0.25
7,100.0,Bajo,566226.16,2820,3,0.85
8,200.0,Bajo,131359.10,743,1,0.20
9,55.0,Medio alto,372696.20,4560,3,0.56


In [32]:

# totales dados
etotal = 196530020.45050645
total_viviendas = 951967

# agrupación y cálculos
resultado = (
    viv_con_ac
    .groupby(['m2', 'estrato'], as_index=False)
    .apply(lambda g: pd.Series({
        'consumo_total_pond': (g['kwh_mes'] * g['factor']).sum(),
        'viviendas_pond': g['factor'].sum()
    }))
    .reset_index(drop=True)
)

# porcentajes
resultado['pct_consumo_total'] = 100 * resultado['consumo_total_pond'] / etotal
resultado['pct_viviendas'] = 100 * resultado['viviendas_pond'] / total_viviendas

# ordenar
resultado = resultado.sort_values('consumo_total_pond', ascending=False)

# print(resultado.columns)
cols = ['m2', 'estrato',  'pct_consumo_total', 'pct_viviendas']
print(resultado[cols].round(1).to_markdown(index=False))


|   m2 | estrato    |   pct_consumo_total |   pct_viviendas |
|-----:|:-----------|--------------------:|----------------:|
|  100 | Medio alto |                 5.4 |             6.7 |
|  100 | Alto       |                 4.1 |             6.2 |
|  150 | Alto       |                 3.7 |             4.5 |
|  150 | Medio alto |                 2.8 |             4.2 |
|  200 | Alto       |                 2.7 |             2   |
|  150 | Medio bajo |                 2.7 |             2.4 |
|  100 | Medio bajo |                 2.6 |             4.1 |
|   75 | Medio alto |                 2.5 |             3.1 |
|  200 | Medio alto |                 2.4 |             2.5 |
|   75 | Medio bajo |                 1.3 |             2.7 |
|   55 | Medio bajo |                 1.3 |             1.8 |
|  200 | Medio bajo |                 0.9 |             0.6 |
|   75 | Alto       |                 0.8 |             1.2 |
|  100 | Bajo       |                 0.3 |             0.3 |
|   55 |

C:\Users\gbv\AppData\Local\Temp\ipykernel_11592\1899583107.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


## Calculo sin aire acondicionado

In [34]:

# totales dados
etotal = 196530020.45050645
total_viviendas = 951967

# agrupación y cálculos
resultado = (
    viv_sin_ac
    .groupby(['m2', 'estrato'], as_index=False)
    .apply(lambda g: pd.Series({
        'consumo_total_pond': (g['kwh_mes'] * g['factor']).sum(),
        'viviendas_pond': g['factor'].sum()
    }))
    .reset_index(drop=True)
)

# porcentajes
resultado['pct_consumo_total'] = 100 * resultado['consumo_total_pond'] / etotal
resultado['pct_viviendas'] = 100 * resultado['viviendas_pond'] / total_viviendas


resultado = resultado.sort_values('consumo_total_pond', ascending=False)
cols = ['m2', 'estrato',  'pct_consumo_total', 'pct_viviendas']
print(resultado[cols].round(1).to_markdown(index=False))


|   m2 | estrato    |   pct_consumo_total |   pct_viviendas |
|-----:|:-----------|--------------------:|----------------:|
|  100 | Medio alto |                 4.4 |             7.9 |
|  100 | Medio bajo |                 4.2 |             7.5 |
|   75 | Medio bajo |                 3.3 |             6.4 |
|  150 | Medio bajo |                 2.9 |             4.8 |
|   55 | Medio bajo |                 2.4 |             4.2 |
|  200 | Medio alto |                 2.3 |             2.7 |
|  150 | Medio alto |                 2.2 |             3.7 |
|  200 | Medio bajo |                 2   |             2.2 |
|  100 | Alto       |                 1.7 |             2.9 |
|   75 | Medio alto |                 1.6 |             3.7 |
|  150 | Alto       |                 1.3 |             1.9 |
|   55 | Medio alto |                 1.3 |             1.8 |
|   75 | Alto       |                 0.9 |             1.6 |
|  200 | Alto       |                 0.5 |             1.3 |
|   30 |

C:\Users\gbv\AppData\Local\Temp\ipykernel_11592\1719583142.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
